In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
from TP1.KalmanFilter import KalmanFilter

In [8]:
MOT = pd.read_csv("ADL-Rundle-6\det\Yolov5s\det.csv")
MOT.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1695,386,136,342,0.760947,-1,-1,-1
1,1,-1,255,459,103,246,0.676300,-1,-1,-1
2,1,-1,1885,380,34,202,0.628092,-1,-1,-1
3,1,-1,2,306,81,614,0.575901,-1,-1,-1
4,1,-1,1284,463,74,195,0.575504,-1,-1,-1


In [9]:
# Code CC CV depuis internet https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] <= bb1['x2']
    assert bb1['y1'] <= bb1['y2']
    assert bb2['x1'] <= bb2['x2']
    assert bb2['y1'] <= bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [10]:
def GetDictFromDF(liste,pos):
    liste = liste.iloc[pos].to_list()
    return {"x1":liste[2] ,"y1":liste[3] , "x2":liste[2]  + liste[4] ,"y2":liste[3] + liste[5]}


def getFromKalman(liste,pos,xy):
    liste = liste.iloc[pos].to_list()
    return {"x1":xy[0] ,"y1":xy[1] , "x2":xy[0]  + liste[4] ,"y2":xy[1] + liste[5]}
    

In [11]:
def make_square(matrix):
    rows, cols = matrix.shape
    if rows == cols:
        return matrix
    max_dim = max(rows, cols)
    square_matrix = np.zeros((max_dim, max_dim), dtype=matrix.dtype)
    square_matrix[:rows, :cols] = matrix
    return square_matrix

In [13]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Initialisation
CurrentFrame = 2
df = []
debug = True
nbFrames = len(MOT["frame"].unique())  # Nombre total de frames dans les données
posID = 0

# Initialisation des ID pour les objets de la première image
IDList = [1] * len(MOT[MOT["frame"] == CurrentFrame - 1])
for i in range(1, len(IDList)):
    IDList[i] += IDList[i - 1]

# Initialisation des Kalman Filters
KalList = []
for i in IDList:
    di = GetDictFromDF(MOT[MOT["frame"] == CurrentFrame - 1], i - 1)
    x, y = di["x1"], di["x2"]
    kf = KalmanFilter(1, x, y, 1, 1, 1)  # Initialisation du Kalman Filter
    KalList.append((kf, i))  # Stockage du Kalman Filter et de l'ID correspondant

IDMax = max(IDList)
IDList = [IDList]  # Liste des ID sur toutes les frames

# Boucle principale pour suivre les objets sur toutes les frames
while CurrentFrame != nbFrames:
    # Objets suivis dans la frame actuelle et suivante
    CurrentTrackedList = MOT[MOT["frame"] == CurrentFrame - 1]
    NewTrackedList = MOT[MOT["frame"] == CurrentFrame]
    
    # Matrice des IoU (Intersection over Union)
    MatriceIoU = []
    for lig in range(len(CurrentTrackedList)):
        MatriceIoU.append([])
        value1forIoU = getFromKalman(CurrentTrackedList, lig, KalList[lig][0].predict())
        for col in range(len(NewTrackedList)):
            value2forIoU = GetDictFromDF(NewTrackedList, col)
            iou_value = get_iou(value1forIoU, value2forIoU)
            MatriceIoU[lig].append(iou_value)
    
    # Conversion en tableau NumPy
    print(MatriceIoU)
    MatriceIoU = np.array(MatriceIoU, dtype=np.float64)  # Force un tableau homogène
    Mshape = MatriceIoU.shape
    MatriceIoU = make_square(MatriceIoU)  # Rend la matrice carrée si nécessaire

    # Calcul de la distance inversée pour l'affectation
    matcpy = MatriceIoU.copy()
    maximum = matcpy.max()
    matcpy = maximum - matcpy
    poxlist = linear_sum_assignment(matcpy)[1]  # Résolution avec Hungarian Algorithm

    # Gestion des cas où la matrice a plus de lignes que de colonnes
    if Mshape[0] > Mshape[1]:
        pl = []
        for i in range(len(poxlist)):
            if MatriceIoU[i][poxlist[i]] != 0:
                pl.append(poxlist[i])
            else:
                pl.append(-1)  # Aucun objet associé
        poxlist = pl

    # Mise à jour des nouveaux ID
    newIDList = [0] * len(poxlist)
    for i in range(len(poxlist)):
        if poxlist[i] < len(IDList[posID]) and poxlist[i] != -1:
            newIDList[i] = IDList[posID][poxlist[i]]
        else:
            newIDList[i] = IDMax
            IDMax += 1

    # Affichage des résultats intermédiaires pour debug
    if debug:
        print("---------")
        print(f"Frame: {CurrentFrame}")
        print("New ID List:", newIDList)
        print("Poxlist:", poxlist)
        print("---------")
    
    # Ajout des nouveaux IDs et passage à la frame suivante
    posID += 1
    IDList.append(newIDList)
    CurrentFrame += 1

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, matrix([[0.37524366]]), 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


C:\Users\lylia\AppData\Local\Temp\ipykernel_22348\1357659864.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  iou = intersection_area / float(bb1_area + bb2_area - intersection_area)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 8) + inhomogeneous part.

In [11]:
import cv2
import os

frames_folder = "ADL-Rundle-6/img1/"

# Lire toutes les frames du dossier
frames = sorted([os.path.join(frames_folder, f) for f in os.listdir(frames_folder) if f.endswith('.jpg')])

# Lire et afficher chaque frame avec des rectangles
framepos = 1
for frame_path in frames:
    # Charger l'image
    frame = cv2.imread(frame_path)
    height, width = frame.shape[:2]
    
    rectangles = []
    for i in range(len(MOT[MOT["frame"] == framepos])):
        dicte = GetDictFromDF(MOT[MOT["frame"] == framepos],i)
        rectangles.append(dicte)
        KLList[i].update(dicte["x1"],dicte["y1"])
    
    assert(len(rectangles) == len(MOT[MOT["frame"] == framepos]))
    # Ajouter des rectangles
    posrec = 0
    for rect in rectangles:
        x1, y1, x2, y2 = int(rect["x1"]), int(rect["y1"]), int(rect["x2"]), int(rect["y2"]) # Regarder par rapport au axes
        color = (0,255,0)
        thickness = 2
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        font_thickness = 3
        text_color = (255, 0, 0)
        if framepos >= len(IDList) -1:
            break
        print(framepos)
        print(posrec)
        print(len(rectangles))
        print(IDList[framepos-1])
        print("---------")
        title = str(IDList[framepos-1][posrec])
        cv2.putText(frame, title, (x1, y1), font, font_scale, text_color, font_thickness, cv2.LINE_AA)
        cv2.putText(frame, str(framepos), (30, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str(len(MOT[MOT["frame"] == framepos])), (30, 70), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        posrec += 1
    
    framepos +=1
    # Afficher la frame
    cv2.imshow('Video Frame', frame)
    
    a = 40
    if framepos == 50:
        break
    # Attendre 30 ms pour simuler un FPS de ~30
    if cv2.waitKey(a) & 0xFF == ord('q'):  # Quitter avec 'q'
        break

cv2.destroyAllWindows()


NameError: name 'KLList' is not defined